In [1]:
!pip install --upgrade --quiet  transformers --quiet

In [1]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 256},
)

In [4]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256)
hf = HuggingFacePipeline(pipeline=pipe)

In [2]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'


In [7]:
from langchain_core.prompts import PromptTemplate

template = """give me best possible text generation"""
prompt = PromptTemplate.from_template(template)

chain = prompt | hf

question = "how not to think about something?"

print(chain.invoke({"question": question}))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


give me best possible text generation. I think I've found a way to use the tools on your behalf

So I'll try to be clear on my use of the tools you've written in the guides and the notes. I thought it'd be fun to try and clarify things, so don't hesitate to leave a note if you have any doubts (I would just be happy to assist so everyone can know I'm good at all my writing and editing!!)

I'm just a guy so please feel free to use the code if you would like ;)

Enjoyed this guide. It has been a while though and I would love to see any feedback you guys have for the tools in the guide so please feel free to send them out. Hope you enjoyed the guide too, as a bonus this could be a HUGE help for anyone looking through tutorials in the past.

I'd also love to see all the suggestions on how to edit the guides as well as other things about your work. So really don't let this discourage you ;) Enjoy

EDIT: It seems you can use the tooltips to make a small number of adjustments without adding an

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_mistralai import MistralAIEmbeddings

# Load
url = "https://lilianweng.github.io/posts/2023-06-23-agent/"
loader = WebBaseLoader(url)
docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=100
)
all_splits = text_splitter.split_documents(docs)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Index
vectorstore = Chroma.from_documents(
    documents=all_splits,
    collection_name="rag-chroma",
    embedding=embedding_function,
)
retriever = vectorstore.as_retriever()

In [5]:
### Retrieval Grader 

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

prompt = PromptTemplate(
    template="""You are a grader assessing relevance of a retrieved document to a user question. \n 
    """,
    input_variables=["question", "document"],
)

retrieval_grader = prompt | hf | JsonOutputParser()
question = "agent memory"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

OutputParserException: Invalid json output: You are a grader assessing relevance of a retrieved document to a user question. 
 
     

The key is to let the user decide how an event should be seen

for one time to another, then move to the action and the context-sensitive items

and move to the action and and move to the Context menu.

Example 1 :

If you would like to create an interface, use the menu item at the right

as its key,

The key has the value "Keyless Access"

is the Context menu.

Example 2 :

If you would like to create shortcuts, use the item:

If you would like to create the user interface, use item:

Or use the menu item as key:

It is really easy. 

A very common style of tool, is the

function getcontextmenu ( name ) {

return getcontextmenu (( ContextMenu ) name); }

A very common design for

The tool is shown in some examples below.

Example 3 :

If you would like to have a simple interface, use the menu item at the left

as its key,

If you would like to have a s

In [12]:
### Generate

from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt
# prompt = hub.pull("rlm/rag-prompt")
template = """Question: {question}

Answer: explore"""
prompt = PromptTemplate.from_template(template)


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

question = "what is agent"

# Chain
rag_chain = prompt | hf | StrOutputParser()

# Run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

Question: what is agent?

Answer: explore agents.

If you just want to discover what is real about the world and the process of it, use agents.

You can ask specific questions about the environment, what is real about your situation, what is just about you.

You can also ask questions: what is your experience with you, who are you, and what is your experience with some things.

You should also check out agent-learn.com as all the resources are great and the questions that you come to when you ask them. This site just helps you learn more about agents.

Here is a list of agents that are helpful and there are no resources that could give you all of the specific answers you are looking for.

There are also other resources that can help you learn more about the whole process of this question in depth.

We recommend using agents and having it learn for you.

Here is what I am interested in with this question.

Do you have questions about how to make this question better? Would you be happy 

In [13]:
# Prompt 
re_write_prompt = PromptTemplate(
    template="""You a question re-writer that converts an input question to a better version that is optimized \n 
     for vectorstore retrieval. Look at the initial and formulate an improved question. \n
     Here is the initial question: \n\n {question}. Improved question with no preamble: \n """,
    input_variables=["generation", "question"],
)

question_rewriter = re_write_prompt | hf | StrOutputParser()
question_rewriter.invoke({"question": question})
# print(question_rewriter)

first=PromptTemplate(input_variables=['question'], template='You a question re-writer that converts an input question to a better version that is optimized \n \n     for vectorstore retrieval. Look at the initial and formulate an improved question. \n\n     Here is the initial question: \n\n {question}. Improved question with no preamble: \n ') middle=[HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7f3045a82020>, model_kwargs={}, pipeline_kwargs={'max_new_tokens': 256})] last=StrOutputParser()
